#LIBRERIAS

In [70]:
pip.main(['install', 'matplotlib'])
pip.main(['install', 'plotly'])
pip.main(['install', 'plotly_express'])
pip.main(["install","openpyxl"])
import pandas as pd
import numpy as np
import plotly.express as px
from datetime import datetime, timedelta
import streamlit as st

#METODOS

In [ ]:
def reemplazos(v):
  v=str(v)
  v=v.replace(',','')
  return v

In [71]:
def nombres(a,b,c):
  return str(a)+' '+str(b)+' '+str(c)

In [72]:
def maximizador_pantalla(DF):
  TEMPORAL=DF.groupby('numero_socio').max()
  TEMPORAL['numero_socio']=TEMPORAL.index
  TEMPORAL=TEMPORAL.reset_index(drop=True)
  TEMPORAL=TEMPORAL[['numero_socio','paso']]
  TEMPORAL=TEMPORAL.rename(columns={'paso':'PASO_MAXIMO'})
  return TEMPORAL

In [73]:
def maximizador_fecha(DF):
  TEMPORAL=DF.groupby('numero_socio').max()
  TEMPORAL['numero_socio']=TEMPORAL.index
  TEMPORAL=TEMPORAL.reset_index(drop=True)
  TEMPORAL=TEMPORAL[['numero_socio','FECHA']]
  TEMPORAL=TEMPORAL.rename(columns={'FECHA':'FECHA_MAXIMA'})
  return TEMPORAL

In [74]:
def ULTIMA_PANTALLA(v):
  if v==0:
    return 'PANTALLA HOME'
  elif v==1:
    return 'DATOS PERSONALES'
  elif v==2:
    return 'DATOS ADICIONALES'
  elif v==3:
    return 'VERIFICACION CELULAR Y ACEPTACION BURO'
  elif v==4:
    return 'VERIFICACION INE'
  elif v==5:
    return 'COMPROBANTE DE DOMICILIO'
  elif v==6:
    return 'VALIDACION DE SELFI'
  elif v==7:
    return 'VIDEO_SELFIE'
  elif v==8:
    return 'PENDIENTE'
  elif v==9:
    return 'BENEFICIOS DE SER SOCIO, PARAMETRIZAR Y CONFIRMAR OFERTA'
  elif v==10:
    return 'NO ENCONTRADO'
  elif v==11:
    return 'MESA DE CONTROL'

#ARCHIVO

In [75]:
DF=pd.read_excel('ONBOARDING ACTIVA 2 AL 19 ENERO.xlsx',sheet_name='Hoja1',usecols=['nombre_s','apellido_paterno','apellido_materno','numero_socio','celular','evaluado','aprobado','mensaje_rechazo','paso_proceso','Icc','BcScore','EstimadorIngresos','fecha_created_at'])
DF=DF.rename(columns={'paso_proceso':'paso','fecha_created_at':'FECHA'})

LIMPIEZA

In [76]:
DF=DF.merge(maximizador_pantalla(DF),on='numero_socio',how='left')
DF=DF.drop('paso',axis=1)
DF=DF.merge(maximizador_fecha(DF),on='numero_socio',how='left')
DF=DF.drop('FECHA',axis=1)
DF=DF.rename(columns={'PASO_MAXIMO':'paso'})
DF=DF.rename(columns={'FECHA_MAXIMA':'FECHA'})
DF['NOMBRE']=list(map(nombres,DF['nombre_s'],DF['apellido_paterno'],DF['apellido_materno']))
DF['ultima_pantalla']=list(map(ULTIMA_PANTALLA,DF['paso']))
DF=DF.drop_duplicates()
DF=DF[['NOMBRE','numero_socio','celular','evaluado','aprobado','mensaje_rechazo','ultima_pantalla','paso','Icc','BcScore','EstimadorIngresos','FECHA']]

In [77]:
DF=DF.fillna('VACIO')

#CODIGO

In [ ]:
st.title("PROTOTIPO FUNNELL")

##FUNNEL

In [ ]:
st.markdown(f' FUNNEL')

In [98]:
LISTA_PASOS=[]
LISTA_VALORES=[]

In [99]:
for i in range(0,12):
  LISTA_PASOS.append(i)
  LISTA_VALORES.append(len(DF[DF['paso']>=i]))

In [100]:
LISTA_PASOS=list(map(ULTIMA_PANTALLA,LISTA_PASOS))

In [101]:
data = dict(
    number=LISTA_VALORES,
    stage=LISTA_PASOS)
fig = px.funnel(data, x='number', y='stage')
fig.show()
st.plotly_chart(fig) # de esta forma se va a mostrar el dataframe en Streamlit

##BARRAS DE ACTIVIDAD

In [ ]:
st.markdown(f' AFLUENCIA DIARIA EN EL UNBORDING 2.0')

In [83]:
fecha_minima=DF['FECHA'].min()
fecha_maxima=DF['FECHA'].max()
fechas=[]
dias_a_sumar = 1
fechita=fecha_minima


while fechita<=fecha_maxima:
  fechas.append(fechita)
  fechita = fechita + timedelta(days=dias_a_sumar)

In [84]:
ACTIVIDAD=[]
for i in range(0,len(fechas)-1):
  ACTIVIDAD.append(len(DF[(DF['FECHA']>=fechas[i])&(DF['FECHA']<=fechas[i+1])]))

In [85]:
# Datos de muestra
x = fechas[0:len(ACTIVIDAD)]
y = ACTIVIDAD
df = pd.DataFrame(list(zip(x,y)),columns=['FECHA','FRECUENCIA'])
fig = px.bar(df, x = 'FECHA', y = 'FRECUENCIA')
fig.show()
st.plotly_chart(fig) # de esta forma se va a mostrar el dataframe en Streamlit

DeltaGenerator()

In [86]:
# Datos
#x = fechas[0:len(ACTIVIDAD)]
#y = ACTIVIDAD

# Gráfico de barras
#fig, ax = plt.subplots(figsize=(20,10))
#ax.bar(x=x, height=y)

# Mostrar el gráfico
#plt.show()

##EN PROCESO

In [ ]:
st.markdown(f' CAIDOS')

In [87]:
TEMPORAL=DF[(DF['aprobado']==1)&(DF['evaluado']==1)&(DF['mensaje_rechazo']=='VACIO')]
TEMPORAL=TEMPORAL[['NOMBRE','numero_socio','celular','ultima_pantalla','paso','FECHA']]
TEMPORAL=TEMPORAL.sort_values(by='paso')
TEMPORAL=TEMPORAL.reset_index(drop=True)
TEMPORAL['NUMERO_SOCIO']=list(map(reemplazos,TEMPORAL['NUMERO_SOCIO']))
TEMPORAL['CELULAR']=list(map(reemplazos,TEMPORAL['CELULAR']))
TEMPORAL=TEMPORAL.rename(columns={'numero_socio':'NUMERO_SOCIO','celular':'CELULAR','ultima_pantalla':'ULTIMA_PANTALLA','paso':'PASO'})
TEMPORAL.to_csv('EN PROCESO.csv')
st.write(TEMPORAL)

,NOMBRE,NUMERO_SOCIO,CELULAR,ULTIMA_PANTALLA,PASO,FECHA
0,DULCE MARIA TORRES SOTO,492830,4433022740,PANTALLA HOME,0,2023-12-11 16:20:44.787
1,ISMAEL GARCIA ESTRELLA,547617,7123516450,DATOS PERSONALES,1,2023-12-02 12:01:09.470
2,LAURA CALDERAS ANGUIANO,549134,4434090177,DATOS PERSONALES,1,2023-12-08 09:35:49.700
3,DANIELA MICHEL ORTIZ REYES,549197,4436866385,DATOS PERSONALES,1,2023-12-08 11:45:20.820
4,ALFONSO TREJO ABAD,540764,4471721596,DATOS PERSONALES,1,2024-01-17 14:22:51.037
5,ALAN YAIR EZEQUIEL MOLINA,554709,4433721194,DATOS ADICIONALES,2,2024-01-03 08:35:09.120
6,JUAN ESPINOZA SANDOVAL,547111,7531444211,DATOS ADICIONALES,2,2024-01-15 19:09:40.617
7,RAQUEL TELLO MORENO,551007,4431139601,VERIFICACION CELULAR Y ACEPTACION BURO,3,2023-12-16 11:47:09.723
8,ARISBETH DIAZ FIGUEROA,507121,4434599474,VERIFICACION INE,4,2024-01-15 18:35:40.040
9,ALICIA GUZMAN MARQUEZ,478230,4433518085,COMPROBANTE DE DOMICILIO,5,2023-12-16 09:59:12.457


In [88]:
PROCESOS=TEMPORAL.pivot_table( ['FECHA'], ['PASO','NOMBRE','NUMERO_SOCIO','CELULAR'])
#PROCESOS=PROCESOS.sort_values(by='FECHA',ascending=False)
PROCESOS.to_csv('PROCESOS_PIVOTE.csv')
PROCESOS

FECHA
PASO NOMBRE                           NUMERO_SOCIO CELULAR                                 
0    DULCE MARIA TORRES SOTO          492830       4433022740 2023-12-11 16:20:44.787000064
1    ALFONSO TREJO ABAD               540764       4471721596 2024-01-17 14:22:51.036999936
     DANIELA MICHEL ORTIZ REYES       549197       4436866385 2023-12-08 11:45:20.820000000
     ISMAEL GARCIA ESTRELLA           547617       7123516450 2023-12-02 12:01:09.470000128
     LAURA CALDERAS ANGUIANO          549134       4434090177 2023-12-08 09:35:49.700000000
2    ALAN YAIR EZEQUIEL MOLINA        554709       4433721194 2024-01-03 08:35:09.120000000
     JUAN ESPINOZA SANDOVAL           547111       7531444211 2024-01-15 19:09:40.616999936
3    RAQUEL TELLO MORENO              551007       4431139601 2023-12-16 11:47:09.723000064
4    ARISBETH DIAZ FIGUEROA           507121       4434599474 2024-01-15 18:35:40.040000000
5    ALICIA GUZMAN MARQUEZ            478230       4433518085 2023-12-16 09:59:12.456999936
7    JUAN GARCIA ESCOBEDO             544216       4523436601 2023-12-30 07:57:32.447000064
9    ARLETH YESENIA GODINEZ VELAZQUEZ 549892       7713851772 2023-12-11 15:39:00.036999936

##APROBADOS

In [89]:
TEMPORAL=DF[(DF['aprobado']==1)&(DF['evaluado']==1)&(DF['mensaje_rechazo']=='N')]#&(DF['paso']==11)]
TEMPORAL=TEMPORAL[['NOMBRE','numero_socio','celular','ultima_pantalla','paso','FECHA']]
TEMPORAL=TEMPORAL.sort_values(by='paso')
TEMPORAL=TEMPORAL.reset_index(drop=True)
TEMPORAL=TEMPORAL.rename(columns={'numero_socio':'NUMERO_SOCIO','celular':'CELULAR','ultima_pantalla':'ULTIMA_PANTALLA','paso':'PASO'})
TEMPORAL=TEMPORAL.drop('PASO',axis=1)
TEMPORAL['NUMERO_SOCIO']=list(map(reemplazos,TEMPORAL['NUMERO_SOCIO']))
TEMPORAL['CELULAR']=list(map(reemplazos,TEMPORAL['CELULAR']))
TEMPORAL.to_csv('APROBADOS.csv')
st.write(TEMPORAL)

,NOMBRE,NUMERO_SOCIO,CELULAR,ULTIMA_PANTALLA,PASO,FECHA
0,ALONDRA ITZEL BOYZO SOTO,1012030,4438667413,MESA DE CONTROL,11,2023-12-01 20:06:40.443


##RECHAZADO

In [90]:
TEMPORAL=DF[(DF['aprobado']==0)&(DF['evaluado']==1)]
TEMPORAL=TEMPORAL[['NOMBRE','numero_socio','celular','mensaje_rechazo','Icc','BcScore','EstimadorIngresos','paso','FECHA']]
TEMPORAL=TEMPORAL.sort_values(by='mensaje_rechazo')
TEMPORAL=TEMPORAL.reset_index(drop=True)
TEMPORAL=TEMPORAL.rename(columns={'numero_socio':'NUMERO_SOCIO','celular':'CELULAR','mensaje_rechazo':'MENSAJE_RECHAZO','Icc':'ICC','BcScore':'BCSCORE','EstimadorIngresos':'ESTIMADOR_INGRESOS','paso':'PASO'})
df['NUMERO_SOCIO']=list(map(reemplazos,df['NUMERO_SOCIO']))
df['CELULAR']=list(map(reemplazos,df['CELULAR']))
TEMPORAL.to_csv('RECHAZADO.csv')st.write(TEMPORAL)

,NOMBRE,NUMERO_SOCIO,CELULAR,MENSAJE_RECHAZO,ICC,BCSCORE,ESTIMADOR_INGRESOS,PASO,FECHA
0,GLORIA PRIETO MEDINA,547985,4641596446,"El socio supera la edad, no procede el credito",-3,-8,11,1,2023-12-04 15:54:18.880
1,GIBRAN ALBERTO LAGUNAS MOZQUEDA,558918,4191311409,El socio tiene domicilio en una region no oper...,6,552,22,4,2024-01-17 16:35:52.607
2,ERIKA IBARRA ENCISO,547618,3318698167,El socio tiene domicilio en una region no oper...,5,649,15,1,2023-12-02 22:56:06.297
3,ANGEL ADRIAN AGUIRRE ESQUIVEL,557401,4775501262,El socio tiene domicilio en una region no oper...,6,657,31,1,2024-01-12 11:29:42.937
4,MARTIN LERA GUERRERO,375021,4121714626,El socio tiene domicilio en una region no oper...,6,663,16,1,2023-12-04 10:34:35.053
...,...,...,...,...,...,...,...,...,...
172,MARIA CONCEPCION TORAL ECHEVARRIA,550113,4521129611,No cumple con Score,0,520,13,1,2023-12-13 12:15:13.897
173,MIREYA ANAI PANTOJA CHAVEZ,432934,4523436601,No cumple con Score,5,625,38,1,2024-01-15 16:10:30.017
174,CRISTHIAN GARCIA SANCHEZ,549975,5633609908,No cumple con Score,-3,-8,19,1,2023-12-12 19:59:07.943
175,ROSA ISELA CRUZ ELVIRA,551124,4251236392,No cumple con Score,-3,-5,14,1,2023-12-16 19:54:27.510


In [91]:
RECHAZOS=TEMPORAL.pivot_table(['ICC','BCSCORE','ESTIMADOR_INGRESOS','FECHA'],['PASO','MENSAJE_RECHAZO','NOMBRE','NUMERO_SOCIO','CELULAR'])
RECHAZOS=RECHAZOS.sort_values(by='FECHA',ascending=False)
RECHAZOS.to_csv('RECHAZOS_PIVOTE.csv')
RECHAZOS

BCSCORE  \
PASO MENSAJE_RECHAZO                                    NOMBRE                             NUMERO_SOCIO CELULAR               
4    El socio tiene domicilio en una region no operable GIBRAN ALBERTO LAGUNAS MOZQUEDA    558918       4191311409    552.0   
1    No cumple con Score                                LUIS JESUS MENDEZ ESCALONA         558684       5518221900    520.0   
                                                        MANUEL ARTEMIO LEON GONZALEZ       420959       4423553576    561.0   
                                                        OSCAR ALEJANDRO BARCENAS HERNANDEZ 547296       4771648040    586.0   
     Se encuentra clave(s) de observación               ROSALBA HERNANDEZ BARAJAS          558537       4432660303    658.0   
...                                                                                                                     ...   
     No cumple con Score                                PABLO SERGIO JUAREZ LOPEZ          547350       5622023643     -9.0   
                                                        YAZMIN RAMIREZ LOPEZ               547319       5548190713     -9.0   
                                                        ANAHI BAUTISTA NUÑEZ               547305       7422579578     -7.0   
     ICC < 5                                            ANDRES MAURICIO VAZQUEZ MENDOZA    264968       4141191371    646.0   
     No cumple con Score                                ALEJANDRA SANCHEZ VILLALON         547302       5637224258    578.0   

                                                                                                                    ESTIMADOR_INGRESOS  \
PASO MENSAJE_RECHAZO                                    NOMBRE                             NUMERO_SOCIO CELULAR                          
4    El socio tiene domicilio en una region no operable GIBRAN ALBERTO LAGUNAS MOZQUEDA    558918       4191311409                22.0   
1    No cumple con Score                                LUIS JESUS MENDEZ ESCALONA         558684       5518221900                14.0   
                                                        MANUEL ARTEMIO LEON GONZALEZ       420959       4423553576                17.0   
                                                        OSCAR ALEJANDRO BARCENAS HERNANDEZ 547296       4771648040                25.0   
     Se encuentra clave(s) de observación               ROSALBA HERNANDEZ BARAJAS          558537       4432660303                20.0   
...                                                                                                                                ...   
     No cumple con Score                                PABLO SERGIO JUAREZ LOPEZ          547350       5622023643                10.0   
                                                        YAZMIN RAMIREZ LOPEZ               547319       5548190713                10.0   
                                                        ANAHI BAUTISTA NUÑEZ               547305       7422579578                16.0   
     ICC < 5                                            ANDRES MAURICIO VAZQUEZ MENDOZA    264968       4141191371                25.0   
     No cumple con Score                                ALEJANDRA SANCHEZ VILLALON         547302       5637224258                14.0   

                                                                                                                                           FECHA  \
PASO MENSAJE_RECHAZO                                    NOMBRE                             NUMERO_SOCIO CELULAR                                    
4    El socio tiene domicilio en una region no operable GIBRAN ALBERTO LAGUNAS MOZQUEDA    558918       4191311409 2024-01-17 16:35:52.607000064   
1    No cumple con Score                                LUIS JESUS MENDEZ ESCALONA         558684       5518221900 2024-01-17 11:40:07.160000000   
                                                        MANUEL ARTEMIO LEON G

##INDEFINIDO

In [92]:
TEMPORAL=DF[(DF['aprobado']==0)&(DF['evaluado']==0)]
TEMPORAL=TEMPORAL[['NOMBRE','numero_socio','celular','mensaje_rechazo','paso','FECHA']]
TEMPORAL=TEMPORAL.sort_values(by='mensaje_rechazo')
TEMPORAL=TEMPORAL.reset_index(drop=True)
TEMPORAL=TEMPORAL.rename(columns={'numero_socio':'NUMERO_SOCIO','celular':'CELULAR','mensaje_rechazo':'MENSAJE_RECHAZO','paso':'PASO'})
TEMPORAL.to_csv('INDEFINIDO.csv')
TEMPORAL

,NOMBRE,NUMERO_SOCIO,CELULAR,MENSAJE_RECHAZO,PASO,FECHA
